In [1]:
import numpy as np

import pandas as pd
import pandas_ta as ta # pandas technical analysis
import pandas_datareader.data as web
pd.options.display.float_format = "{:,.2f}".format

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-white")
plt.rcParams["font.size"] = 11
plt.rcParams["lines.linestyle"] = "-"
plt.rcParams["figure.dpi"] = 1500
plt.rcParams["figure.figsize"] = (15,6)

from datetime import datetime

import plotly.graph_objs as go
import plotly as py

import yfinance as yf

# momentum indicators
from ta.momentum import RSIIndicator
from ta.momentum import ROCIndicator
from ta.momentum import StochasticOscillator
from ta.momentum import WilliamsRIndicator

# trend indicators
from ta.trend import MACD
from ta.trend import EMAIndicator
from ta.trend import WMAIndicator
from ta.trend import SMAIndicator

# volatility indicators
from ta.volatility import BollingerBands
from ta.volatility import AverageTrueRange

# volume indicators
from ta.volume import money_flow_index
from ta.volume import OnBalanceVolumeIndicator

In [2]:
class TechnicalIndicators:
    """
    This is an example of the TechIndicator class.
    """

    window = 14 # class or static variable can be referred through a class but not directly through an instance
    
    def __init__(self, close, high, low, volume):
        self.close = close
        self.high = high
        self.low = low
        self.volume = volume
        self.technical_indicators = None
        
    def set_technical_indicators(self):
        self.technical_indicators = pd.DataFrame()
       
        self.technical_indicators["Close"] = self.close
        self.technical_indicators["High"] = self.high
        self.technical_indicators["Low"] = self.low
        self.technical_indicators["Volume"] = self.volume
        
# momentum indicators
    def rate_change(self):
        """
        Rate of Change. Momentum Indicator
        """
        closing_price = self.close
        df = self.technical_indicators
        
        roc = ROCIndicator(
                    close = closing_price,
                    window = TechnicalIndicators.window,
                    fillna = False
                    )
        
        df["ROC"] = roc.roc()

        return df    
    
    def rsi(self):
        """
        The relative strength index. Momentum Indicator.
        """
        closing_price = self.close
        df = self.technical_indicators
        
        rsi = RSIIndicator(
                        close = closing_price, 
                        window = TechnicalIndicators.window,
                        fillna = False
                          )
        
        df["RSI"] = rsi.rsi()
        
        return df
    
    def stochastic_oscillator(self):
        """
        Stochastic Oscillator. Momentum Indicator.
        """
        df = self.technical_indicators
        
        stoch_osci = StochasticOscillator(
            close = self.close,
            high = self.high,
            low = self.low,
            window = TechnicalIndicators.window,
            smooth_window = 3,
            fillna = False
             )
        
        df["Stochastic Oscillator"] = stoch_osci.stoch()
    
        return df
    
    def WilliamsR(self):
        """
        WilliamsR Indicator. Momentum Indicator.
        """
        df = self.technical_indicators

        williamsR = WilliamsRIndicator(
                                    high = self.high,
                                    low = self.low,
                                    close = self.close,
                                    lbp = TechnicalIndicators.window,
                                    fillna = False
                                      )
    
        df["Williams %R Indicator"] = williamsR.williams_r()
    
        return df
    
# trend indicators
    def moving_average(self):
        """
        Moving Average.
        """
        df = self.technical_indicators
        days = [5,10,20]
        
        for i in days:
            col_name = f"MA for {i} days"
            df[col_name] = self.close.rolling(window = i).mean()
            
        return df
    
    def macd(self):
        """
        The moving average convergence/divergence.
        """
        df = self.technical_indicators
        closing_price = self.close
        ema_26 = 26
        ema_12 = 12
        signal = 9
        macd = MACD(
            closing_price, 
            window_fast = ema_12, 
            window_slow = ema_26,
            window_sign = signal,
            fillna = False
                    )
        df["MACD"] = macd.macd()
        
        return df

    def ema(self):
        """
        The exponential moving average. Trend indicator.
        """
        df = self.technical_indicators
    
        ema = EMAIndicator(
                close = self.close,
                window = TechnicalIndicators.window,
                fillna = False
                    )
        
        df["Exponential moving average"] = ema.ema_indicator()
        
        return df
    
    def wma(self):
        """
        Weighted moving average. Trend indicator.
        """
        df = self.technical_indicators
        
        wma = WMAIndicator(
                close = self.close, 
                window = TechnicalIndicators.window, 
                fillna = False
                )
        
        df["Weighted moving average"] = wma.wma()
        
        return df

# volatility indicators
    def bollinger_bands(self):
        """
        Bollinger bands. Volatility Indicator.
        """
        closing_price = self.close
        df = self.technical_indicators
        
        bbands = BollingerBands(
                            close = closing_price,
                            window = TechnicalIndicators.window,
                            window_dev = 2,
                            fillna= False
                               )

        df["Bollinger Bands Middle"] = bbands.bollinger_mavg()
        df["Bollinger Bands Lower"] = bbands.bollinger_lband()
        df["Bollinger Bands Upper"] = bbands.bollinger_hband()
        
        return df
    
    def average_true_range(self):
        """
        Average true range. Volatility Indicator.
        """
        df = self.technical_indicators
        
        atr = AverageTrueRange(
                high = self.high,
                low = self.low,
                close = self.close,
                window = TechnicalIndicators.window,
                fillna = False
                        )
        
        df["Average True Range"] = atr.average_true_range()
        
        return df

# volume indicators
    def on_balance_volume(self):
        """
        On Balance Volume. Volume Indicator.
        """
        df = self.technical_indicators
        
        on_bal_vol = OnBalanceVolumeIndicator(
                                close = self.close,
                                volume = self.volume,
                                fillna = False
                                  )
        
        df["On Balance Volume"] = on_bal_vol.on_balance_volume()
        
        return df
 
    def money_flow_index(self):
        """
        Money flow index. Volume Indicator.
        """
        df = self.technical_indicators
        
        mfi = money_flow_index(
                high = self.high,
                low = self.low,
                close = self.close,
                volume = self.volume,
                window = TechnicalIndicators.window, 
                fillna = False
                        )
        
        df["Money Flow Index"] = mfi
        
        return df

In [3]:
end = datetime.today()
start = datetime(end.year-2, end.month, end.day)
ticker = web.DataReader("^GSPC", "yahoo", start, end)

In [4]:
t = TechnicalIndicators(ticker["Close"], ticker["High"], ticker["Low"], ticker["Volume"])
t.set_technical_indicators()

if __name__ == "__main__":
    t.rsi()
    t.rate_change()
    t.stochastic_oscillator()
    t.WilliamsR()

    t.macd()
    t.ema()
    t.wma()
    t.moving_average()

    t.bollinger_bands()
    t.average_true_range()

    t.on_balance_volume()
    t.money_flow_index()

In [5]:
class MovAvg(TechnicalIndicators):
    """
    Class MovAvg inherits attributes and methods from Technical Indicators class.
    """
    s_window = 20
    l_window = 50 
    
    def __init__(self, close, high, low, volume, mov_avg):
                               
        super().__init__(close, high, low, volume)
        self.mov_avg = mov_avg
        
    def name_cols(self):
        """
        Create column names.
        """
        s_window_col = str(MovAvg.s_window) + "_" + self.mov_avg
        l_window_col = str(MovAvg.l_window) + "_" + self.mov_avg
        
        return s_window_col, l_window_col
    
    def strategy(self):
        """
        Create moving average strategy.
        """
        
        df = self.technical_indicators
        
        s_window_col, l_window_col = self.name_cols()
        
        if self.mov_avg == "SMA":
            df[s_window_col] = self.close.rolling(
                                                window = MovAvg.s_window, 
                                                min_periods = 1, 
                                                center = False
                                                ).mean()
            df[l_window_col] = self.close.rolling(
                                                window = MovAvg.l_window, 
                                                min_periods = 1, 
                                                center = False
                                                ).mean()
            
        elif self.mov_avg == "EMA":
            df[s_window_col] = self.close.ewm(
                                            span = MovAvg.s_window, 
                                            adjust = False
                                            ).mean()
            
            df[l_window_col] = self.close.ewm(
                                            span = MovAvg.l_window,
                                            adjust = False
                                            ).mean()
    
        df["Signal"] = 0.0
        df["Signal"] = np.where(df[s_window_col] > df[l_window_col], 1.0, 0.0)
        df["Position"] = df["Signal"].diff()
        
        return df
    
    def plot(self):
        
        df = self.technical_indicators
        
        s_window_col, l_window_col = self.name_cols()
        
        plt.tick_params(axis = "both", labelsize = 10)
        
        df["Close"].plot(color = "k", lw = 0.5, label = "Closing price")
        df[s_window_col].plot(color = "b", lw = 0.5, label = s_window_col)
        df[l_window_col].plot(color = "g", lw = 0.5, label = l_window_col)
        
        plt.plot(
            df[df["Position"] == 1].index,
            df[s_window_col][df["Position"] == 1],
            "^",
            markersize = 8,
            color = "k",
            alpha = 0.7,
            label = "Buy"
                )
     
        plt.plot(
            df[df["Position"] == -1].index,
            df[s_window_col][df["Position"] == -1],
            "v",
            markersize = 8,
            color = "r",
            alpha = 0.7,
            label = "Sell"
                )
        
        plt.xlabel("Date", fontsize = 10)
        plt.ylabel("Price in USD", fontsize = 10)
        plt.title(str(self.mov_avg) + " Golden Cross Strategy")
        plt.legend(loc = "best")

In [6]:
m = MovAvg(ticker["Close"], ticker["High"], ticker["Low"], ticker["Volume"], "SMA")

if __name__ == "__main__":
    m.set_technical_indicators()
    m.name_cols()
    m.strategy()
    m.plot()